This notebook looks to replicate the tutorial material from databrick's own tutorials, only using open source libraries and tools.  
  
Below is a list of the tools used.

http://192.168.1.123:8998  
Livy Server: http://192.168.1.123:8998/ui  
Spark Master: http://192.168.1.123:8080/  
Spark Magic: https://github.com/jupyter-incubator/sparkmagic/blob/master/examples/Magics%20in%20IPython%20Kernel.ipynb  

Let's load up sparkmagic to be able to communicate without spark cluster

In [7]:
%load_ext sparkmagic.magics

The sparkmagic.magics extension is already loaded. To reload it, use:
  %reload_ext sparkmagic.magics


Now let's configure our connection

In [16]:
%manage_spark

TWFnaWNzQ29udHJvbGxlcldpZGdldChjaGlsZHJlbj0oVGFiKGNoaWxkcmVuPShNYW5hZ2VTZXNzaW9uV2lkZ2V0KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGJyLz4nKSwgSEJveChjaGlsZHLigKY=


Let's read the data used in the tutorial

In [9]:
%%spark
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val marketsFile = sc.textFile("file:///home/churtado/notebooks/data/farmers_market.csv")

val taxes2013 = spark.read.format("csv").option("header", "true").load("file:///home/churtado/notebooks/data/zipcodes.csv")
taxes2013.createOrReplaceTempView("taxes2013")
val markets = spark.read.format("csv").option("header", "true").load("file:///home/churtado/notebooks/data/farmers_market.csv")
markets.createOrReplaceTempView("markets")

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@503b36f6
marketsFile: org.apache.spark.rdd.RDD[String] = file:///home/churtado/notebooks/data/farmers_market.csv MapPartitionsRDD[1] at textFile at <console>:26
taxes2013: org.apache.spark.sql.DataFrame = [STATEFIPS: string, STATE: string ... 112 more fields]
markets: org.apache.spark.sql.DataFrame = [FMID: string, MarketName: string ... 57 more fields]


We can use sql magic to view the data

In [10]:
%%spark -c sql
SHOW TABLES

,database,tableName,isTemporary
0,NaT,markets,True
1,NaT,taxes2013,True


In [11]:
%%spark -c sql -o df_markets 
SELECT * FROM markets

,FMID,MarketName,Website,Facebook,city,County,State,zip,Season1Date,Season1Time,...,Tofu,WildHarvested,updateTime,street,OtherMedia,Location,Twitter,Youtube,Season3Date,Season3Time
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,...,N,N,2017-06-20 22:43:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,...,N,N,2017-06-21 17:15:01,6975 Ridge Road,NaN,NaN,NaN,NaN,NaN,NaN
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,Six Mile,NaN,South Carolina,29682,NaN,NaN,...,N,N,2013-01-01 00:00:00,106 S. Main Street,NaN,NaN,NaN,NaN,NaN,NaN
3,1010691,10th Steet Community Farmers Market,NaN,NaN,Lamar,Barton,Missouri,64759,04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,...,N,N,2014-10-28 09:49:46,10th Street and Poplar,http://agrimissouri.com/mo-grown/grodetail.php...,NaN,NaN,NaN,NaN,NaN
4,1002454,112st Madison Avenue,NaN,NaN,New York,New York,New York,10029,July to November,Tue:8:00 am - 5:00 pm;Sat:8:00 am - 8:00 pm;,...,N,N,2012-03-01 10:38:22,112th Madison Avenue,NaN,Private business parking lot,NaN,NaN,NaN,NaN
5,1011100,12 South Farmers Market,http://www.12southfarmersmarket.com,12_South_Farmers_Market,Nashville,Davidson,Tennessee,37204,05/05/2015 to 10/27/2015,Tue: 3:30 PM-6:30 PM;,...,N,N,2015-05-01 10:40:56,3000 Granny White Pike,@12southfrmsmkt,NaN,@12southfrmsmkt,NaN,NaN,NaN
6,1009845,125th Street Fresh Connect Farmers' Market,http://www.125thStreetFarmersMarket.com,https://www.facebook.com/125thStreetFarmersMarket,New York,New York,New York,10027,06/10/2014 to 11/25/2014,Tue: 10:00 AM-7:00 PM;,...,N,N,2014-04-07 16:32:01,"163 West 125th Street and Adam Clayton Powell,...",Instagram--> 125thStreetFarmersMarket,Federal/State government building grounds,https://twitter.com/FarmMarket125th,NaN,NaN,NaN
7,1005586,12th & Brandywine Urban Farm Market,NaN,https://www.facebook.com/pages/12th-Brandywine...,Wilmington,New Castle,Delaware,19801,05/16/2014 to 10/17/2014,Fri: 8:00 AM-11:00 AM;,...,N,N,2014-04-03 15:43:31,12th & Brandywine Streets,https://www.facebook.com/delawareurbanfarmcoal...,"On a farm from: a barn, a greenhouse, a tent, ...",NaN,NaN,NaN,NaN
8,1008071,14&U Farmers' Market,NaN,https://www.facebook.com/14UFarmersMarket,Washington,District of Columbia,District of Columbia,20009,05/03/2014 to 11/22/2014,Sat: 9:00 AM-1:00 PM;,...,N,N,2014-04-05 13:49:04,1400 U Street NW,NaN,Other,https://twitter.com/14UFarmersMkt,NaN,NaN,NaN
9,1012710,14th & Kennedy Street Farmers Market,NaN,https://www.facebook.com/14KennnedyFarmersMarket/,Washington,District of Columbia,District of Columbia,20011,04/09/2016 to 11/19/2016,Sat: 9:00 AM-1:00 PM;,...,N,N,2016-07-20 11:16:24,"5500 Colorado Avenue, NW",instagram:14kenfm,NaN,14KenFM,NaN,NaN,NaN


We can now use pandas dataframes for the data

In [12]:
df_markets.head(3)

,FMID,MarketName,Website,Facebook,city,County,State,zip,Season1Date,Season1Time,...,Tofu,WildHarvested,updateTime,street,OtherMedia,Location,Twitter,Youtube,Season3Date,Season3Time
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,...,N,N,2017-06-20 22:43:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,...,N,N,2017-06-21 17:15:01,6975 Ridge Road,NaN,NaN,NaN,NaN,NaN,NaN
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,Six Mile,NaN,South Carolina,29682,NaN,NaN,...,N,N,2013-01-01 00:00:00,106 S. Main Street,NaN,NaN,NaN,NaN,NaN,NaN


We can visualize the data as well

In [13]:
from autovizwidget.widget.utils import display_dataframe
display_dataframe(df_markets)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oSFRNTCh2YWx1ZT11J1R5cGU6JyksIEJ1dHRvbihkZXNjcmlwdGlvbj11J1RhYmxlJywgbGF5b3V0PUxheW91dCh3aWR0aD11JzcwcHgnKSzigKY=


Output()

AutoVizWidget()

Now we're going to clean up the data with SQL

In [67]:
%%spark -c sql
DROP TABLE IF EXISTS cleaned_taxes

""


In [68]:
%%spark -c sql -o df_cleaned_taxes

CREATE TABLE cleaned_taxes AS
SELECT state, int(zipcode / 10) as zipcode, 
  int(mars1) as single_returns, 
  int(mars2) as joint_returns, 
  int(numdep) as numdep, 
  double(A02650) as total_income_amount,
  double(A00300) as taxable_interest_amount,
  double(a01000) as net_capital_gains,
  double(a00900) as biz_net_income
FROM taxes2013

""


We want to look at avg income per state

In [62]:
%%spark -c sql
DROP TABLE IF EXISTS income_state

""


In [63]:
%%spark -c sql

CREATE TABLE income_state AS
SELECT state, 
sum(total_income_amount) / sum(1) as total_income
FROM cleaned_taxes
group by state

""


In [69]:
%%spark -c sql -o df_income_state

select * from income_state

,state,total_income
0,OK,56894.028083
1,NY,156471.204280
2,TX,155419.941538
3,VA,113140.714405
4,MS,51964.313514
5,CT,199774.300505
6,NE,35791.415808
7,WA,152404.440428
8,ND,29765.335673
9,CO,142955.932830


Let's try to plot income by state using plotly

In [33]:
import plotly
plotly.tools.set_credentials_file(username='churtado', api_key='iaMRV6ydU9Ove5Yfy0R7')

In [66]:
states = df_income_state["state"]
values = df_income_state["total_income"]

import plotly.plotly as py
import pandas as pd

data = [ dict(
        type='choropleth',
        autocolorscale = True,
        locations = df_income_state['state'],
        z = df_income_state['total_income'].astype(float),
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Total Avg Income")
        ) ]

layout = dict(
        title = 'Map of avg income',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )